# StarbucksDT_land_price crwaling using Selenium

#### To-Do list
##### 1. open the page
##### 2. read csv file → make dataframe
##### 3. extract all DT stores from dataframe and append it
##### 4. write DT's address on search bar
##### 5. crawling price using selenium
##### 6. extract city name from csv file name
##### 7. save file → set file name using location name

In [1]:
#!pip install selenium
# !pip install bs4

In [1]:
from urllib import request
from selenium import webdriver
import pandas as pd
import numpy as np
import bs4
import requests
import re
import csv
from time import sleep
from selenium.common.exceptions import ElementNotVisibleException

## 스타벅스 DT 데이터만 따로 뽑아 저장

In [107]:
# read csv file name
city_name_list = ['서울', '경기', '광주', '대구', '대전', '부산', '울산', '인천', '강원', '경남',  '경북', '전남', '전북', '충남', '충북',  '제주', '세종']
i = 0;

while i < len(city_name_list):
    path = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_data/" #저장하고 싶은 경로 앞부분 
    full_path = path + 'starbucks_' + city_name_list[i] +'.csv' #현재 경로 + 파일명
    
    path2 = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_DT_data/"
    full_path2 = path2 + 'starbucks_DT_' + city_name_list[i] +'.csv'
    
    # 전체 지정이 담겨져 있는 csv파일 읽어오기
    store = pd.read_csv(full_path, engine = 'python', encoding = 'utf-8')
    store = store.drop(columns=['Unnamed: 0']) #  Unnamed컬럼 삭제
    
  
   # name에 DT포함되어 있는 데이터프레임 새로 생성
    DT_list = store[store['name'].str.contains("DT")]
    DT_list = DT_list.reset_index(drop=True)
    
    # DT지점만 따로 csv파일로 생성
    DT_list.to_csv(full_path2, header=True, index=True, encoding='euc-kr')
    
    i = i+1

## 홈페이지에서 스타벅스 DT의 땅값 알아보기

In [2]:
# 다운받은 chromedriver의 path설정과 크롤링주소 설정
path=r"C:\Users\Kavin\Downloads\chromedriver_win32\chromedriver"
driver=webdriver.Chrome(path)
driver.get("http://www.onnara.go.kr/OnnaraServiceMA/onnaraCommon/mainPage.do")
sleep(10)

In [3]:
driver.find_element_by_xpath('/html/body/div[3]/div[7]/div/button').click()
sleep(5)

In [4]:
driver.find_element_by_xpath('//*[@id="guide"]/div[1]/a').click()
sleep(5)

In [5]:
# price값에서 ','제외한 숫자데이터만 뽑아옵니다
def get_numeric(data):
    match = re.search('>(.+)<', data)
    if match:
        return int(match.group(1).replace(',',''))    
    return None

# 전체 지역 공시지가 크롤링 코드

In [186]:
# csv 파일을 읽어오고 데이터프레임에 넣은 후 공시지가 컬럼 새로 추가

city_name_list = ['서울', '경기', '광주', '대구', '대전', '부산', '울산', '인천', '강원', '경남',  '경북', '전남', '전북', '충남', '충북',  '제주', '세종']
price_list = []
i = 0

while i < len(city_name_list):
    path = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_DT_data/" #저장하고 싶은 경로 앞부분 
    full_path = path + 'starbucks_DT_' + city_name_list[i] +'.csv' #현재 경로 + 파일명

    # 전체 지정이 담겨져 있는 csv파일 읽어오기
    store = pd.read_csv(full_path, engine = 'python', encoding = 'euc-kr')
    store = store.drop(columns=['Unnamed: 0']) #  Unnamed컬럼 삭제
    j=0
    
    while j < len(store):
        
        address_keyword = store['address'][j]
        

        # 검색창에 주소 입력
        address_search = driver.find_element_by_xpath('//*[@id="totalSerarch"]')
        address_search.clear()
        address_search.send_keys(address_keyword)
        sleep(2)
        # 찾기버튼 누르기
        search = driver.find_element_by_xpath('//*[@id="headerMap"]/section[1]/a[2]')
        search.click()
        sleep(5)
        # 리스트에서 첫번째 항목 
        click_address = driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[1]/div/ul/li/ul')
        click_address.click()
        sleep(5)
        
        # 공시지가 가져오기
        source = driver.page_source
        bs = bs4.BeautifulSoup(source,'lxml')
        price_list.append(bs.select('#infoJiga'))


        j = j + 1
        
    
    store['price'] = price_list
    store['price'] = store['price'].astype('str')
    store['price'] = store['price'].apply(get_numeric)

       
    path2 = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_DT_price/"
    full_path2 = path2 + 'starbucks_DT_price_' + city_name_list[i] +'.csv'
    store.to_csv(full_path2, header=True, index=True, encoding='euc-kr')
    

    
    i = i + 1

WebDriverException: Message: unknown error: Element <a href="#" title="지도검색" class="btn_go">...</a> is not clickable at point (437, 31). Other element would receive the click: <div class="sweet-overlay" tabindex="-1" style="opacity: 1; display: block;"></div>
  (Session info: chrome=68.0.3440.106)
  (Driver info: chromedriver=2.41.578737 (49da6702b16031c40d63e5618de03a32ff6c197e),platform=Windows NT 10.0.17134 x86_64)


## 추가해야할 부분

1) 지도 이동 불가능하다는 경고창 에러 해결

2) 주소 검색시 결과값 나오지 않는 에러 해결

3) 정확한 지도 데이터 가져오기(정확한 도로명주소로 검색하여 비교 후 색인)

### 1) 지도 이동 불가능 시 0값 처리 후 크롤링하기

In [1]:
# apply to original code

# csv 파일을 읽어오고 데이터프레임에 넣은 후 공시지가 컬럼 새로 추가

city_name_list = ['서울', '경기', '광주', '대구', '대전', '부산', '울산', '인천', '강원', '경남',  '경북', '전남', '전북', '충남', '충북',  '제주', '세종']
price_list = []
i = 0

while i < len(city_name_list):
    path = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_DT_data/" #저장하고 싶은 경로 앞부분 
    full_path = path + 'starbucks_DT_' + city_name_list[i] +'.csv' #현재 경로 + 파일명

    # 전체 지정이 담겨져 있는 csv파일 읽어오기
    store = pd.read_csv(full_path, engine = 'python', encoding = 'euc-kr')
    store = store.drop(columns=['Unnamed: 0']) #  Unnamed컬럼 삭제
   
    j=0
    
    while j < len(store):
        
        address_keyword = store['address'][j]
        

        # 검색창에 주소 입력
        address_search = driver.find_element_by_xpath('//*[@id="totalSerarch"]')
        address_search.clear()
        address_search.send_keys(address_keyword)
        sleep(2)
        # 찾기버튼 누르기
        search = driver.find_element_by_xpath('//*[@id="headerMap"]/section[1]/a[2]')
        search.click()
        sleep(5)
        # 리스트에서 첫번째 항목 
        click_address = driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[1]/div/ul/li/ul')
        click_address.click()
        sleep(5)
        
        try:
            if len(driver.find_elements_by_class_name('sa-confirm-button-container')) > 0:
                driver.find_element_by_xpath('/html/body/div[3]/div[7]/div/button').click()
                price_list.append('')
                sleep(2)
        except ElementNotVisibleException as e:
                source = driver.page_source
                bs = bs4.BeautifulSoup(source,'lxml')
                price_list.append(bs.select('#infoJiga'))
                sleep(2)
        print(price_list[j])    
        j = j + 1
        
    
    store['price'] = price_list
    store['price'] = store['price'].astype('str')
    store['price'] = store['price'].apply(get_numeric)

       
    path2 = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_DT_price/"
    full_path2 = path2 + 'starbucks_DT_price_' + city_name_list[i] +'.csv'
    store.to_csv(full_path2, header=True, index=True, encoding='euc-kr')
    

    
    i = i + 1

NameError: name 'pd' is not defined

### 2) 주소 검색 시 값이 나오지 않을 때 에러 처리

In [35]:
# apply to original code

# csv 파일을 읽어오고 데이터프레임에 넣은 후 공시지가 컬럼 새로 추가

city_name_list = ['서울', '경기', '광주', '대구', '대전', '부산', '울산', '인천', '강원', '경남',  '경북', '전남', '전북', '충남', '충북',  '제주', '세종']
price_list = []
i = 0

while i < len(city_name_list):
    price_list.clear()

    path = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_DT_data/" #저장하고 싶은 경로 앞부분 
    full_path = path + 'starbucks_DT_' + city_name_list[i] +'.csv' #현재 경로 + 파일명

    # 전체 지정이 담겨져 있는 csv파일 읽어오기
    store = pd.read_csv(full_path, engine = 'python', encoding = 'euc-kr')
    store = store.drop(columns=['Unnamed: 0']) #  Unnamed컬럼 삭제
   
    j=0
    
    while j < len(store):
        
        address_keyword = store['address'][j]
        

        # 검색창에 주소 입력
        address_search = driver.find_element_by_xpath('//*[@id="totalSerarch"]')
        address_search.clear()
        address_search.send_keys(address_keyword)
        sleep(2)
        # 찾기버튼 누르기
        search = driver.find_element_by_xpath('//*[@id="headerMap"]/section[1]/a[2]')
        search.click()
        sleep(5)
        # 리스트에서 첫번째 항목 

        if len(driver.find_elements_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[1]/div/ul/li[1]')) > 0:
            click_address = driver.find_element_by_xpath('//*[@id="searchMenu"]/div[1]/ul/li[1]/div/div/div/div[2]/ul/li[2]/div/div[1]/div/ul/li[1]/ul')
            click_address.click()
            sleep(8)
            
            try:
                if len(driver.find_elements_by_class_name('sa-confirm-button-container')) > 0:
                    driver.find_element_by_xpath('/html/body/div[3]/div[7]/div/button').click()
                    price_list.append('')
                    sleep(2)
            except ElementNotVisibleException as e:
                    source = driver.page_source
                    bs = bs4.BeautifulSoup(source,'lxml')
                    price_list.append(bs.select('#infoJiga'))
                    sleep(2)

        else:
            price_list.append('')
            sleep(2)
            
        print(price_list[j])
        j = j + 1
        
    
    store['price'] = price_list
    store['price'] = store['price'].astype('str')
    store['price'] = store['price'].apply(get_numeric)

       
    path2 = "C:/Users/Kavin/Projects/Starbucks_Selenium_Crawler/Starbucks_DT_price/"
    full_path2 = path2 + 'starbucks_DT_price_' + city_name_list[i] +'.csv'
    store.to_csv(full_path2, header=True, index=True, encoding='euc-kr')
    
    i = i + 1

[<em class="letter" id="infoJiga">1,591,000</em>]
[<em class="letter" id="infoJiga">1,288,000</em>]
[<em class="letter" id="infoJiga">436,000</em>]
[<em class="letter" id="infoJiga">1,712,000</em>]
[<em class="letter" id="infoJiga">1,478,000</em>]
[<em class="letter" id="infoJiga">950,000</em>]

[<em class="letter" id="infoJiga">765,300</em>]
[<em class="letter" id="infoJiga">352,000</em>]
[<em class="letter" id="infoJiga">655,200</em>]
[<em class="letter" id="infoJiga">808,000</em>]
[<em class="letter" id="infoJiga">1,060,000</em>]
[<em class="letter" id="infoJiga">675,000</em>]
[<em class="letter" id="infoJiga">1,113,000</em>]


[<em class="letter" id="infoJiga">1,453,000</em>]
[<em class="letter" id="infoJiga">894,400</em>]
[<em class="letter" id="infoJiga">285,800</em>]
[<em class="letter" id="infoJiga">795,400</em>]
[<em class="letter" id="infoJiga">1,650,000</em>]
[<em class="letter" id="infoJiga">2,397,000</em>]
[<em class="letter" id="infoJiga">906,900</em>]
[<em class="letter"